In [1]:
import torch
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2Model
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [2]:


# Load and preprocess data
def load_data(file_path):
    data = pd.read_csv(file_path)
    texts = data['text'].tolist()
    scores = data['score'].tolist()
    return texts, scores

In [3]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Tokenize and create BERT embeddings
def create_bert_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained(
        'bert-base-uncased',
        output_hidden_states = True,
    )


    embeddings = []
    num_layers = 10
    for text in tqdm(texts, desc="Creating BERT Embeddings"):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        
        
        hidden_states = torch.stack(outputs.hidden_states[-num_layers:])
#         embeddings.append(outputs.pooler_output.detach().numpy().flatten())
        embedding = torch.cat([hidden_states[i] for i in range(num_layers)], dim=-1)
        embedding = torch.mean(embedding, dim=1).squeeze().detach().numpy()
        embeddings.append(embedding)


    return embeddings



def create_gpt2_embeddings(texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2Model.from_pretrained('gpt2')
    model = model.to(device)

    embeddings = []
    for text in tqdm(texts, desc="Creating GPT-2 Embeddings"):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state

        # Use the mean of the last hidden states as the embedding
        embedding = torch.mean(last_hidden_states, dim=1).squeeze().detach().cpu().numpy()
        embeddings.append(embedding)

    return embeddings

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def get_embeddings(texts, model_name):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)

    embeddings = []

    for text in tqdm(texts):
        # Tokenize input text and obtain embeddings
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract the embeddings from the last layer
        last_hidden_states = outputs.last_hidden_state
        # Average pooling to get a fixed-size embedding for the entire text
        avg_pooling = torch.mean(last_hidden_states, dim=1)
        # Convert tensor to numpy array
        avg_pooling = avg_pooling.cpu().numpy()
        # Append the embedding to the list
        embeddings.append(avg_pooling)

    return np.concatenate(embeddings, axis=0)

models = [
    'xlm-mlm-enfr-1024',
    'distilbert-base-cased',
    'bert-base-uncased',
    'roberta-base',
    'cardiffnlp/twitter-roberta-base-sentiment',
    'xlnet-base-cased',
    
    
#     'ctrl',
#     'transfo-xl-wt103',
    'bert-base-cased',
    'xlm-roberta-base',
#     'openai-gpt',
#     'gpt2'
]


def get_all_embeddings(texts):
    all_embeddings = []

    for model_name in tqdm(models):
        print(f"calculating embeddings for {model_name}")
        embeddings = get_embeddings(texts, model_name)
        all_embeddings.append(embeddings)

    return np.concatenate(all_embeddings, axis=1)




In [13]:

# Define the MLP model
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()
        self.l1 = nn.Sequential(
            
            # BERT 10
#             nn.Linear(7680, 2048),
#             nn.ReLU(),
#             nn.Linear(2048, 1024),
#             nn.ReLU(),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.Linear(512, 256),   
#             nn.ReLU(),
#             nn.Linear(256, 128),   
#             nn.ReLU(),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, 1),
            
            
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )


    def forward(self, x):
        x = self.l1(x)
        return x

In [14]:

# Train MLP model
def train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()

        model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_rmse = mean_squared_error(val_labels, val_preds, squared=False)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation RMSE: {val_rmse:.4f}")


In [ ]:

# Load and preprocess training data
train_texts, train_scores = load_data('train.csv')


# Split data into training and validation sets
train_texts, val_texts, train_scores, val_scores = train_test_split(
    train_texts, train_scores, test_size=0.2, random_state=42
)


embedding_function = get_all_embeddings

# Create BERT embeddings
train_embeddings = embedding_function(train_texts[1:10])
# val_embeddings = embedding_function(val_texts)

  0%|                                                                                                                                   | 0/8 [00:00<?, ?it/s]

calculating embeddings for xlm-mlm-enfr-1024



 12%|███████████████▍                                                                                                           | 1/8 [00:03<00:26,  3.77s/it]

In [11]:

# Convert data to PyTorch tensors
train_data = TensorDataset(torch.tensor(train_embeddings), torch.tensor(train_scores))
val_data = TensorDataset(torch.tensor(val_embeddings), torch.tensor(val_scores))


/tmp/ipykernel_5685/4237676821.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  train_data = TensorDataset(torch.tensor(train_embeddings), torch.tensor(train_scores))


In [12]:
len(train_embeddings[0])

768

In [32]:


# Create DataLoader
batch_size = 512
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Initialize and train the MLP model
input_size = len(train_embeddings[0])
hidden_size = 256
output_size = 1
# model = MLPModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-6)

num_epochs = 100
train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/100 - Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 101.17it/s]

Epoch 1/100 - Validation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 192.03it/s]


Epoch 1/100, Validation RMSE: 0.3782



Epoch 2/100 - Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 113.34it/s]

  2%|██▍                                                                                                                      | 2/100 [00:00<00:07, 13.97it/s]

Epoch 2/100, Validation RMSE: 0.3782



Epoch 3/100 - Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 110.92it/s]

Epoch 3/100 - Validation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 212.84it/s]


...

Epoch 98/100, Validation RMSE: 0.3780



Epoch 99/100 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 117.87it/s]

Epoch 99/100 - Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 230.60it/s]


Epoch 99/100, Validation RMSE: 0.3780



Epoch 100/100 - Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 132.87it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.31it/s]

Epoch 100/100, Validation RMSE: 0.3780


In [36]:



# Load and preprocess test data
test_texts, _ = load_data('sample_submission.csv')
test_embeddings = create_gpt2_embeddings(test_texts)

# Make predictions on test data
test_data = TensorDataset(torch.tensor(test_embeddings))
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


Creating GPT-2 Embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:01<00:00,  4.11it/s]


In [37]:
model

MLPModel(
  (l1): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [38]:

model.eval()
test_preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader, desc="Generating Test Predictions"):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        test_preds.extend(outputs.cpu().numpy())


test_preds = [x[0] for x  in test_preds]

        
# Write predictions to the output CSV file
output_df = pd.DataFrame({'text': test_texts, 'score': test_preds})
output_df.to_csv('gpt2.csv', index=False)


Generating Test Predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 113.00it/s]
